# SageMaker 엔드포인트(Endpoint)

이전에 훈련한 모델을 배포하려면 Sagemaker 엔드포인트를 생성해야 합니다. 엔드포인트는 추론을 수행하는 데 사용할 수 있는 호스팅된 예측 서비스입니다.

## 모델 찾기

이 노트북은 저장된 모델이 있는 경우 사용합니다. 최근 `%store %` 매직 커맨드를 사용하는 훈련 예제(예: `pytorch_smdataparallel_mnist_demo.ipynb`)를 실행했다면 다음 코드 셀에서 복원됩니다.

그렇지 않으면 URI를 `model_data` 변수의 모델 파일(.tar.gz 파일)에 전달할 수 있습니다.

왼쪽 탐색 창에서 **Training > Training jobs**을 선택하여 [SageMaker console](https://console.aws.amazon.com/sagemaker/home)을 통해 모델 파일을 찾을 수 있습니다. 최근 훈련 작업을 찾아서 선택한 다음 **Output** 창에서 `s3://` 링크를 찾습니다. 모델의 URI를 수동으로 설정하는 다음 셀에서 model_data 줄의 주석 처리를 제거하세요.

In [ ]:
# Retrieve a saved model from a previous notebook run's stored variable
%store -r model_data

# If no model was found, set it manually here.
# model_data = 's3://sagemaker-us-west-2-XXX/pytorch-smdataparallel-mnist-2020-10-16-17-15-16-419/output/model.tar.gz'

print("Using this model: {}".format(model_data))

## 모델 객체(Model object) 생성

SageMaker SDK의 `PyTorchModel`을 사용하여 모델 객체를 정의하고 `estimator` 및 `entry_point`에서 모델을 전달합니다. 추론을 위한 엔드포인트의 엔트리 포인트(entry point)는 `inference.py`를 출력하는 다음 코드 블록에서 볼 수 있듯이 `model_fn`에 의해 정의됩니다. 이 함수는 모델을 로드하고 가능한 경우 GPU를 사용하도록 설정합니다.

In [ ]:
!pygmentize code/inference.py

In [ ]:
import sagemaker
role = sagemaker.get_execution_role()

from sagemaker.pytorch import PyTorchModel
model = PyTorchModel(model_data=model_data, source_dir='code',
                        entry_point='inference.py', role=role, framework_version='1.6.0', py_version='py3')

### 엔드포인트에 모델 배포

`model.deploy` 함수를 사용하여 `predictor`를 생성합니다. 선택적으로 인스턴스 수와 인스턴스 유형을 모두 변경할 수 있습니다.

In [ ]:
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

## 모델 테스트
테스트셋의 샘플을 사용하여 배포된 모델을 테스트할 수 있습니다.

In [ ]:
# Download the test set
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

test_set = datasets.MNIST('data', download=True, train=False, 
                          transform=transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize((0.1307,), (0.3081,))
                            ]))


# Randomly sample 16 images from the test set
test_loader = DataLoader(test_set, shuffle=True, batch_size=16)
test_images, _ = iter(test_loader).next()

# inspect the images
import torchvision
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

def imshow(img):
    img = img.numpy()
    img = np.transpose(img, (1, 2, 0))
    plt.imshow(img)
    return

# unnormalize the test images for displaying
unnorm_images = (test_images * 0.3081) + 0.1307

print("Sampled test images: ")
imshow(torchvision.utils.make_grid(unnorm_images))

In [ ]:
# Send the sampled images to endpoint for inference
outputs = predictor.predict(test_images.numpy())
predicted = np.argmax(outputs, axis=1)

print("Predictions: ")
print(predicted.tolist())

## Cleanup

추론을 시도하거나 엔드포인트에 대해 다른 작업을 수행할 의도가 없다면 불필요한 과금 방지를 위해 엔드포인트를 삭제해야 합니다.

In [ ]:
predictor.delete_endpoint()